In [3]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from sklearn.model_selection import train_test_split
import keras_tuner as kt
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.metrics import mean_squared_error
import numpy as np
from keras import backend as K
import os

In [9]:
import os
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Sequential
import kerastuner as kt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras import backend as K

# Define the directory where the pickle files are stored
folder_path = '../batched_data_pickle_files/'

# Define the filenames for the pickle files
file_names = ['processed_train_data.pkl', 'processed_train_targets.pkl', 'processed_test_data.pkl', 'true_rul.pkl']

# Loop through each file and load its contents as arrays
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    
    # Read the pickle file
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    
    # Ensure data is a numpy array, if not convert it
    if isinstance(data, np.ndarray):
        globals()[file_name.replace('.pkl', '')] = data
    else:
        globals()[file_name.replace('.pkl', '')] = np.array(data)

print("Processed Train Data Shape:", processed_train_data.shape)
print("Processed Train Target Shape:", processed_train_targets.shape)
print("Processed Test Data Shape:", processed_test_data.shape)
print("True RUL Shape:", true_rul.shape)

# Split data into training and validation sets
processed_train_data, processed_val_data, processed_train_targets, processed_val_targets = train_test_split(
    processed_train_data, processed_train_targets, test_size=0.2, random_state=42
)

def build_model(hp):
    model = Sequential()

    # Fixed input shape (timesteps, features)
    input_shape = processed_train_data.shape[1:]  # (L, X), where L is timesteps and X is features

    # Adding batch_size as a hyperparameter (optional)
    batch_size = hp.Int('batch_size', min_value=32, max_value=128, step=32)

    # Determine number of LSTM layers
    num_lstm_layers = hp.Int('num_lstm_layers', 1, 3)  # Choose between 1 and 3 LSTM layers

    # Adding LSTM layers
    for i in range(num_lstm_layers):
        if i == 0:
            model.add(layers.LSTM(
                units=hp.Int('units_1', min_value=128, max_value=256, step=32),
                input_shape=input_shape,
                return_sequences=True,
                activation="tanh"
            ))
        else:
            model.add(layers.LSTM(
                units=hp.Int(f'units_{i + 1}', min_value=32, max_value=128, step=32),
                return_sequences=True if i < num_lstm_layers - 1 else False,
                activation="tanh"
            ))

    # Determine number of Dense layers
    num_dense_layers = hp.Int('num_dense_layers', 1, 2)  # Choose between 1 and 2 Dense layers

    # Adding Dense layers based on the chosen number of layers
    for j in range(num_dense_layers):
        model.add(layers.Dense(
            units=hp.Int(f'dense_units_{j + 1}', min_value=32, max_value=256, step=32),
            activation="relu"
        ))

    model.add(layers.Dense(1))  # Final output layer

    # Compile Model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2)
        ), 
        loss="mse"
    )
    
    return model

# Hyperparameter tuning with Keras Tuner
tuner = kt.Hyperband(
    build_model,
    objective="val_loss",
    max_epochs=10,
    factor=3,
    directory="hyperparam_tuning_2",
    project_name="lstm_tuning_refined_2"
)

# Run the tuner search
tuner.search(
    processed_train_data, processed_train_targets,
    epochs=10,
    validation_data=(processed_val_data, processed_val_targets),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)],
    verbose=2
)

# Get the best model and evaluate it
best_model = tuner.get_best_models(num_models=1)[0]
val_loss = best_model.evaluate(processed_val_data, processed_val_targets, verbose=0)
print(f"\nBest model validation MSE: {val_loss:.4f}")

# Calculate RMSE on validation data
val_predictions = best_model.predict(processed_val_data)
val_rmse = np.sqrt(mean_squared_error(processed_val_targets, val_predictions))
print(f"Validation RMSE: {val_rmse:.4f}")

# Get the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
print("Best hyperparameters:", best_hyperparameters.values)

# Build and train the best model on the full training data
best_batch_size = best_hyperparameters['batch_size']
best_model = build_model(best_hyperparameters)
best_model.fit(processed_train_data, processed_train_targets, epochs=10, batch_size=best_batch_size)

# Save the best model
best_model.save('best_rul_lstm_model.h5')
print("Best model saved to 'best_rul_lstm_model.h5'")

# Final RMSE on the test data
test_predictions = best_model.predict(processed_test_data)
test_rmse = np.sqrt(mean_squared_error(true_rul, test_predictions))
print(f"Test RMSE: {test_rmse:.4f}")


Processed Train Data Shape: (17731, 30, 14)
Processed Train Target Shape: (17731,)
Processed Test Data Shape: (100, 30, 14)
True RUL Shape: (100,)


In [ ]:
import optuna
import tensorflow as tf
from tensorflow.keras import layers, Sequential
import numpy as np
from sklearn.metrics import mean_squared_error
from keras import backend as K

# Clear any previous sessions in Keras
K.clear_session()

# Define the objective function for Optuna
def objective(trial):
    # Build the model with hyperparameters suggested by Optuna
    model = Sequential()

    # Fixed input shape
    input_shape = (processed_train_data.shape[1], processed_train_data.shape[2])

    # Number of LSTM layers (Optuna will choose a value between 1 and 3)
    num_lstm_layers = trial.suggest_int('num_lstm_layers', 1, 3)

    # Adding LSTM layers based on the number of layers suggested
    for i in range(num_lstm_layers):
        if i == 0:
            # First LSTM layer
            model.add(layers.LSTM(
                units=trial.suggest_int('units_1', min_value=128, max_value=256, step=32),
                input_shape=input_shape,
                return_sequences=True,
                activation="tanh"
            ))
        else:
            # Subsequent LSTM layers
            model.add(layers.LSTM(
                units=trial.suggest_int(f'units_{i + 1}', min_value=32, max_value=128, step=32),
                return_sequences=True if i < num_lstm_layers - 1 else False,
                activation="tanh"
            ))

    # Number of Dense layers (Optuna will choose a value between 1 and 2)
    num_dense_layers = trial.suggest_int('num_dense_layers', 1, 2)

    # Adding Dense layers
    for j in range(num_dense_layers):
        model.add(layers.Dense(
            units=trial.suggest_int(f'dense_units_{j + 1}', min_value=32, max_value=256, step=32),
            activation="relu"
        ))

    model.add(layers.Dense(1))  # Final output layer

    # Learning rate for the Adam optimizer (between 1e-4 and 1e-2)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate),
        loss='mse'
    )

    # Batch size (suggested by Optuna)
    batch_size = trial.suggest_int('batch_size', 32, 128, step=32)

    # Train the model with early stopping
    model.fit(
        processed_train_data, processed_train_targets,
        epochs=10,
        batch_size=batch_size,
        validation_data=(processed_val_data, processed_val_targets),
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)],
        verbose=0
    )

    # Evaluate the model on validation data
    val_predictions = model.predict(processed_val_data)
    val_rmse = np.sqrt(mean_squared_error(processed_val_targets, val_predictions))

    # Return RMSE for optimization (minimize RMSE)
    return val_rmse

# Create an Optuna study for hyperparameter optimization
study = optuna.create_study(direction='minimize')  # Minimize the RMSE (root mean squared error)
study.optimize(objective, n_trials=100)  # Run 100 trials

# Print the best hyperparameters and validation RMSE
print("Best Hyperparameters:", study.best_params)
print("Best Validation RMSE:", study.best_value)

# Build the best model with the best hyperparameters
best_trial = study.best_trial
best_hyperparameters = best_trial.params

# Create the best model using the optimal hyperparameters
best_model = Sequential()

input_shape = (processed_train_data.shape[1], processed_train_data.shape[2])

# Adding LSTM layers based on the best trial's parameters
num_lstm_layers = best_hyperparameters['num_lstm_layers']
for i in range(num_lstm_layers):
    if i == 0:
        best_model.add(layers.LSTM(
            units=best_hyperparameters[f'units_{i + 1}'],
            input_shape=input_shape,
            return_sequences=True,
            activation="tanh"
        ))
    else:
        best_model.add(layers.LSTM(
            units=best_hyperparameters[f'units_{i + 1}'],
            return_sequences=True if i < num_lstm_layers - 1 else False,
            activation="tanh"
        ))

# Adding Dense layers
num_dense_layers = best_hyperparameters['num_dense_layers']
for j in range(num_dense_layers):
    best_model.add(layers.Dense(
        units=best_hyperparameters[f'dense_units_{j + 1}'],
        activation="relu"
    ))

best_model.add(layers.Dense(1))  # Final output layer

# Compile the model with the best learning rate
best_model.compile(
    optimizer=tf.keras.optimizers.Adam(best_hyperparameters['learning_rate']),
    loss='mse'
)

# Train the best model on the full training data
best_model.fit(
    processed_train_data, processed_train_targets,
    epochs=10,
    batch_size=best_hyperparameters['batch_size'],
    validation_data=(processed_val_data, processed_val_targets),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)],
    verbose=1
)

# Evaluate on the test data
test_predictions = best_model.predict(processed_test_data)
test_rmse = np.sqrt(mean_squared_error(true_rul, test_predictions))
print(f"Test RMSE: {test_rmse:.4f}")

# Save the best model to a file
best_model.save('best_rul_lstm_model_optuna.h5')
print("Best model saved to 'best_rul_lstm_model_optuna.h5'")

